## Setting the Database

In [ ]:
## autoreload 
%load_ext autoreload
%autoreload 2

## defining the paths: 
PATH = "./data/clean_data.csv"
SAVE_DIR = "./data/final_df_for_db.csv"

### Loading DataFrame

In [ ]:
## loading helpers 
from utils.helpers import prepare_dataframe

to_use = prepare_dataframe(path = PATH, save_dir = SAVE_DIR)

### Start the Database

Essentially doing this 

1) Database 
2) Connection to DB with action 
3) Ask queries to the system

In [ ]:
## importing the connectors 
from db.connector import DBConnector
## load the environment variables
from db.db_helpers import load_env_vars, clear_db

## import the materials 
from db.Materials import (Material,populate_materials_from_df,
                          add_material,update_material_name,
                          add_material_relationship)

## import the processes 
from db.Processes import (Process,populate_process_from_df)

In [ ]:
## loading environment variables to connect to the database 
env_vars = load_env_vars()
## instantiate the connector 
connector = DBConnector(**env_vars)
## start the connection 
connector._connect()
## close the connection 
#connector._close()
## getting all the Material nodes
#Material.nodes.all()
clear_db()

### Testing the db 

In [ ]:
hg_dict = {"uuid": "M99",
                 "name": "Hg",
                 "quantity": 100, "unit": "ton",
                 "cost": 1000000000,
                 "description":"Material 99 - is pure Silver"} ## the dictionary for the first material

hg,status = add_material(hg_dict,verbose=True)


ur_dict = {"uuid": "M98",
                 "name": "Ur",
                 "quantity": 999, "unit": "ton",
                 "cost": 99999999999,
                 "description":"Material 98 - is pure Uranium, used in Max's Laboratory"} ## the dictionary for the first material

ur,status2 = add_material(ur_dict,verbose=True)

print(Material.nodes.all(),end='\n')
#clear_db()

In [ ]:
clear_db()
## Silver
hg,status = add_material(hg_dict,verbose=False)
## Uranium 
ur,status2 = add_material(ur_dict,verbose=False)
## create a relationship 
rel_status1 = add_material_relationship(hg, ur) ## hg --> ur 
rel_status2 = add_material_relationship(ur, hg) ## ur --> hg 

## update the material name 
old_uuid = "M99"
new_name = 'K'
upname1 = update_material_name(material_id=old_uuid, new_name=new_name)

### Populating the db

In [ ]:
populate_materials_from_df(df=to_use, verbose=True)

In [ ]:
## populate the processes
populate_process_from_df(df=to_use, verbose=True)

In [ ]:
Material.nodes.all()

In [ ]:
Process.nodes.all()

### Queries

In [ ]:
## class for managing the queries to the database 
from db.Query import Query
from neomodel import db


In [38]:
Q.query

'MATCH (n:Material {uuid: "M1"})-[r]-(m) RETURN r, n, m'

In [41]:
## instantiate the object 
Q = Query(db)
## make a material query 
Q.material_query("M1")
print(Q.run_query()[0][0])
## make a process query
Q.process_query("P1")
print(Q.run_query())
## make a subgraph query
Q.subgraph_query("M1","Material")
print(Q.run_query())

[<Node id=100 labels=frozenset({'Material'}) properties={'quantity': -73, 'name': 'M1__', 'uuid': 'M1'}>]
([[<Node id=120 labels=frozenset({'Process'}) properties={'name': 'P1__XX', 'uuid': 'P1'}>]], ['n'])
([[<Relationship id=881 nodes=(<Node id=149 labels=frozenset({'Process'}) properties={'name': 'P9__XX', 'uuid': 'P9'}>, <Node id=100 labels=frozenset({'Material'}) properties={'quantity': -73, 'name': 'M1__', 'uuid': 'M1'}>) type='OUTPUT_MATERIAL' properties={}>, <Node id=100 labels=frozenset({'Material'}) properties={'quantity': -73, 'name': 'M1__', 'uuid': 'M1'}>, <Node id=149 labels=frozenset({'Process'}) properties={'name': 'P9__XX', 'uuid': 'P9'}>], [<Relationship id=871 nodes=(<Node id=148 labels=frozenset({'Process'}) properties={'name': 'P8__XX', 'uuid': 'P8'}>, <Node id=100 labels=frozenset({'Material'}) properties={'quantity': -73, 'name': 'M1__', 'uuid': 'M1'}>) type='OUTPUT_MATERIAL' properties={}>, <Node id=100 labels=frozenset({'Material'}) properties={'quantity': -73,

### Parsing the Query output

In [35]:
from utils.helpers import (parse_subgraph, parsed_subgraph_to_df)

In [39]:
subgraph_mat = Q.run_query()
data,rels = parse_subgraph(subgraph_mat)

In [37]:
df = parsed_subgraph_to_df(data,verbose=True)

INPUT_MATERIAL  for Process P26; M1 --> P26
INPUT_MATERIAL  for Process P15; M1 --> P15
INPUT_MATERIAL  for Process P13; M1 --> P13
INPUT_MATERIAL  for Process P12; M1 --> P12
INPUT_MATERIAL  for Process P1; M1 --> P1
OUTPUT_MATERIAL for Process P9; P9 --> M1
OUTPUT_MATERIAL for Process P8; P8 --> M1
OUTPUT_MATERIAL for Process P7; P7 --> M1
OUTPUT_MATERIAL for Process P28; P28 --> M1
OUTPUT_MATERIAL for Process P24; P24 --> M1
OUTPUT_MATERIAL for Process P21; P21 --> M1
OUTPUT_MATERIAL for Process P20; P20 --> M1
OUTPUT_MATERIAL for Process P19; P19 --> M1
OUTPUT_MATERIAL for Process P18; P18 --> M1
OUTPUT_MATERIAL for Process P11; P11 --> M1


In [40]:
df

,Relationship,From,To
4,INPUT_MATERIAL,M1,P26
10,INPUT_MATERIAL,M1,P15
11,INPUT_MATERIAL,M1,P13
12,INPUT_MATERIAL,M1,P12
14,INPUT_MATERIAL,M1,P1
0,OUTPUT_MATERIAL,P9,M1
1,OUTPUT_MATERIAL,P8,M1
2,OUTPUT_MATERIAL,P7,M1
3,OUTPUT_MATERIAL,P28,M1
5,OUTPUT_MATERIAL,P24,M1
